## Final Output

In [1]:
%matplotlib inline  
import matplotlib as mpl
import matplotlib.pyplot as plt
from statistics import mean, median, variance
plt.rcParams['figure.figsize'] = [10, 5]
import pprint
import math
import tabulate

def get_overheads(file_name):
    data = []
    with open(file_name, 'r') as results:
        for line in results:
            line = line.split(",")
            trip_duration = float(line[4])
            overhead = float(line[6])
            agent = line[7]
            preference = line[8].replace('\r', '').replace('\n', '')
            
            data.append(overhead)
    return data

def get_utilizations(file_name):
    utilizations = []
    line_no = 0
        
    with open(file_name, 'r') as results:
        for line in results:
            line = line.split(",")
            line[len(line)-1] = line[len(line)-1].replace('\r', '').replace('\n', '')
            line_no = line_no + 1
            if line_no == 1:
                edges = line
            else: 
                utilizations.append([float(u) for u in line[1:]])   
    
    streets_data = {}
    for i in range(len(edges)):
        streets_data[edges[i]] = [utilization[i] for utilization in utilizations]

    streets_utilizations = {}
    for key, value in streets_data.iteritems():
        streets_utilizations[key] = mean(value)
    
    return streets_utilizations

def get_wait_times(file_name):
    wait_times = []
    line_no = 0
        
    with open(file_name, 'r') as results:
        for line in results:
            line = line.split(",")
            line[len(line)-1] = line[len(line)-1].replace('\r', '').replace('\n', '')
            line_no = line_no + 1
            if line_no == 1:
                lanes = line
            else: 
                wait_times.append([float(u) for u in line[1:]])   
    
    wait_times_data = {}
    for i in range(len(lanes)):
        wait_times_data[lanes[i]] = [wait_time[i] for wait_time in wait_times]

    lane_wait_times = {}
    for key, value in wait_times_data.iteritems():
        lane_wait_times[key] = mean(value)
    
    return lane_wait_times

lb_b0p0_overhead_csv = "data/lb_b0p0overheads.csv"
lb_b0p0_streets_csv = "data/lb_b0p0streets.csv"
lb_b0p0_waits_csv = "data/lb_b0p0waits.csv"
lb_b0p9_overhead_csv = "data/lb_b0p9overheads.csv"
lb_b0p9_streets_csv = "data/lb_b0p9streets.csv"
lb_b0p9_waits_csv = "data/lb_b0p9waits.csv"
lb_b1p0_overhead_csv = "data/lb_b1p0overheads.csv"
lb_b1p0_streets_csv = "data/lb_b1p0streets.csv"
lb_b1p0_waits_csv = "data/lb_b1p0waits.csv"

lb_b0p0_overhead = get_overheads(lb_b0p0_overhead_csv)
lb_b0p0_streets = get_utilizations(lb_b0p0_streets_csv)
lb_b0p0_waits = get_wait_times(lb_b0p0_waits_csv)

lb_b0p9_overhead = get_overheads(lb_b0p9_overhead_csv)
lb_b0p9_streets = get_utilizations(lb_b0p9_streets_csv)
lb_b0p9_waits = get_wait_times(lb_b0p9_waits_csv)

lb_b1p0_overhead = get_overheads(lb_b1p0_overhead_csv)
lb_b1p0_streets = get_utilizations(lb_b1p0_streets_csv)
lb_b1p0_waits = get_wait_times(lb_b1p0_waits_csv)

lb_overheads = []
lb_overheads.append(lb_b1p0_overhead)
lb_overheads.append(lb_b0p0_overhead)
lb_overheads.append(lb_b0p9_overhead)

lb_utilizations = []
lb_utilizations.append(lb_b1p0_streets)
lb_utilizations.append(lb_b0p0_streets)
lb_utilizations.append(lb_b0p9_streets)

lb_waits = []
lb_waits.append(lb_b1p0_waits)
lb_waits.append(lb_b0p0_waits)
lb_waits.append(lb_b0p9_waits)


labels = []
labels.append("Beta")
labels.append("Median of Trip Overhead")
labels.append("&Delta;")
labels.append("Variance of Street Utilization")
labels.append("&Delta;")
labels.append("Mean Wait time of lanes")
labels.append("&Delta;")

betas = [1.0, 0.0, 0.9]

output = []
line = 0
baseline_o = float()
baseline_u = float()
baseline_w = float()

for i in range(len(betas)):
    if line == 0:
        baseline_o = median(lb_overheads[i])
        baseline_u = variance(lb_utilizations[i].values())
        baseline_w = mean(lb_waits[i].values())
    temp = []
    temp.append(betas[i])
    
    temp.append(median(lb_overheads[i]))
    if(line != 0):
        temp.append(float(((median(lb_overheads[i]) - baseline_o) / baseline_o) * 100))
    else:
        temp.append("(baseline)")
    
    temp.append(variance(lb_utilizations[i].values()))
    if(line != 0):
        temp.append(float(((variance(lb_utilizations[i].values()) - baseline_u) / baseline_u) * 100))
    else:
        temp.append("(baseline)")
    
    temp.append(mean(lb_waits[i].values()))
    if(line != 0):
        temp.append(float(((mean(lb_waits[i].values()) - baseline_w) / baseline_w) * 100))
    else:
        temp.append("(baseline)")
    
    output.append(temp)
    line = line + 1
    
# output
    
table = tabulate.tabulate(output, labels, 'unsafehtml')
table

Beta,Median of Trip Overhead,Δ,Variance of Street Utilization,Δ,Mean Wait time of lanes,Δ
1,1.88001,(baseline),0.00319859,(baseline),20.6293,(baseline)
0,1.89319,0.701164423065,0.00239556,-25.1057212619,19.7422,-4.30011761279
0.9,1.88176,0.0929516775627,0.00285126,-10.8588719721,19.7668,-4.18123133159


In [3]:
# %matplotlib inline  
import matplotlib as mpl
import matplotlib.pyplot as plt
from statistics import mean, median, variance
plt.rcParams['figure.figsize'] = [10, 5]
import pprint
import math
import tabulate

def get_overheads(file_name):
    data = []
    with open(file_name, 'r') as results:
        for line in results:
            line = line.split(",")
            trip_duration = float(line[4])
            overhead = float(line[6])
            agent = line[7]
            preference = line[8].replace('\r', '').replace('\n', '')
            
            data.append(overhead)
    return data

def get_utilizations(file_name):
    utilizations = []
    line_no = 0
        
    with open(file_name, 'r') as results:
        for line in results:
            line = line.split(",")
            line[len(line)-1] = line[len(line)-1].replace('\r', '').replace('\n', '')
            line_no = line_no + 1
            if line_no == 1:
                edges = line
            else: 
                utilizations.append([float(u) for u in line[1:]])   
    
    streets_data = {}
    for i in range(len(edges)):
        streets_data[edges[i]] = [utilization[i] for utilization in utilizations]

    streets_utilizations = {}
    for key, value in streets_data.iteritems():
        streets_utilizations[key] = mean(value)
    
    return streets_utilizations

def get_wait_times(file_name):
    wait_times = []
    line_no = 0
        
    with open(file_name, 'r') as results:
        for line in results:
            line = line.split(",")
            line[len(line)-1] = line[len(line)-1].replace('\r', '').replace('\n', '')
            line_no = line_no + 1
            if line_no == 1:
                lanes = line
            else: 
                wait_times.append([float(u) for u in line[1:]])   
    
    wait_times_data = {}
    for i in range(len(lanes)):
        wait_times_data[lanes[i]] = [wait_time[i] for wait_time in wait_times]

    lane_wait_times = {}
    for key, value in wait_times_data.iteritems():
        lane_wait_times[key] = mean(value)
    
    return lane_wait_times

tl_b0p0_overhead_csv = "data/tl_b0p0overheads.csv"
tl_b0p0_streets_csv = "data/tl_b0p0streets.csv"
tl_b0p0_waits_csv = "data/tl_b0p0waits.csv"
tl_b0p9_overhead_csv = "data/tl_b0p9overheads.csv"
tl_b0p9_streets_csv = "data/tl_b0p9streets.csv"
tl_b0p9_waits_csv = "data/tl_b0p9waits.csv"
tl_b1p0_overhead_csv = "data/tl_b1p0overheads.csv"
tl_b1p0_streets_csv = "data/tl_b1p0streets.csv"
tl_b1p0_waits_csv = "data/tl_b1p0waits.csv"

tl_b0p0_overhead = get_overheads(tl_b0p0_overhead_csv)
tl_b0p0_streets = get_utilizations(tl_b0p0_streets_csv)
tl_b0p0_waits = get_wait_times(tl_b0p0_waits_csv)

tl_b0p9_overhead = get_overheads(tl_b0p9_overhead_csv)
tl_b0p9_streets = get_utilizations(tl_b0p9_streets_csv)
tl_b0p9_waits = get_wait_times(tl_b0p9_waits_csv)

tl_b1p0_overhead = get_overheads(tl_b1p0_overhead_csv)
tl_b1p0_streets = get_utilizations(tl_b1p0_streets_csv)
tl_b1p0_waits = get_wait_times(tl_b1p0_waits_csv)

tl_overheads = []
tl_overheads.append(tl_b1p0_overhead)
tl_overheads.append(tl_b0p0_overhead)
tl_overheads.append(tl_b0p9_overhead)

tl_utilizations = []
tl_utilizations.append(tl_b1p0_streets)
tl_utilizations.append(tl_b0p0_streets)
tl_utilizations.append(tl_b0p9_streets)

tl_waits = []
tl_waits.append(tl_b1p0_waits)
tl_waits.append(tl_b0p0_waits)
tl_waits.append(tl_b0p9_waits)

labels = []
labels.append("Beta")
labels.append("Median of Trip Overhead")
labels.append("&Delta;")
labels.append("Variance of Street Utilization")
labels.append("&Delta;")
labels.append("Mean Wait time of lanes")
labels.append("&Delta;")

output = []
line = 0
baseline_o = float()
baseline_u = float()
baseline_w = float()
for i in range(len(betas)):
    if line == 0:
        baseline_o = median(tl_overheads[i])
        baseline_u = variance(tl_utilizations[i].values())
        baseline_w = mean(tl_waits[i].values())
    temp = []
    temp.append(betas[i])
    
    temp.append(median(tl_overheads[i]))
    if(line != 0):
        temp.append(float(((median(tl_overheads[i]) - baseline_o) / baseline_o) * 100))
    else:
        temp.append("(baseline)")
    
    temp.append(variance(tl_utilizations[i].values()))
    if(line != 0):
        temp.append(float(((variance(tl_utilizations[i].values()) - baseline_u) / baseline_u) * 100))
    else:
        temp.append("(baseline)")
    
    temp.append(mean(tl_waits[i].values()))
    if(line != 0):
        temp.append(float(((mean(tl_waits[i].values()) - baseline_w) / baseline_w) * 100))
    else:
        temp.append("(baseline)")
    
    output.append(temp)
    line = line + 1
    
# output
    
table = tabulate.tabulate(output, labels, 'unsafehtml')
table

Beta,Median of Trip Overhead,Δ,Variance of Street Utilization,Δ,Mean Wait time of lanes,Δ
1,1.8609,(baseline),0.00392844,(baseline),36.0913,(baseline)
0,1.86714,0.33504501328,0.00301312,-23.2997207748,31.6344,-12.349008953
0.9,1.87148,0.568232091233,0.00364982,-7.09232001092,33.8534,-6.20079135358
